In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import calendar
import re
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

# 2. Rebuild missing data

In [2]:
localidadCodDictionaryNuse = {1:'USAQUEN',
                              2:'CHAPINERO',
                              3:'SANTA FE',
                              4:'SAN CRISTOBAL',
                              5:'USME',
                              6:'TUNJUELITO',
                              7:'BOSA',
                              8:'KENNEDY',
                              9:'FONTIBON',
                              10:'ENGATIVA',
                              11:'SUBA',
                              12:'BARRIOS UNIDOS',
                              13:'TEUSAQUILLO',
                              14:'LOS MARTIRES',
                              15:'ANTONIO NARIÑO',
                              16:'PUENTE ARANDA',
                              17:'CANDELARIA',
                              18:'RAFAEL URIBE URIBE',
                              19:'CIUDAD BOLIVAR',
                              20:'SUMAPAZ',
                              99:'SIN LOCALIZACION'}

## Methods to rebuild

In [3]:
import import_ipynb
import ws_address_siedco
from selenium.common.exceptions import TimeoutException
import re
import unidecode

importing Jupyter notebook from ws_address_siedco.ipynb


In [4]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [5]:
tags = ["Dirección ingresada: ","Dirección encontrada: ","Tipo dirección: ","Código postal: ","Sector catastral: ",
        "UPZ: ","Localidad: ","Latitud: ","Longitud: ","CHIP: "]
def parse_address_ws(ws_result):
    location = {}
    for idx in range(len(tags)-1):
        if tags[idx] == 'Longitud: ':
            location[tags[idx].replace(': ','')] = re.match(r'^.+(Longitud: )(.\d+.\d+).+$',ws_result).group(2)
        else:
            location[tags[idx].replace(': ','')] = find_between(ws_result,tags[idx],tags[idx+1])
    return location

In [6]:
def assign_upz(original_df,index,UPZ_ws_field):
    original_df.at[index,'COD_UPZ'] = find_between(UPZ_ws_field, '(', ')')
    original_df.at[index,'UPZ'] = find_between(UPZ_ws_field, '', ' (')

In [7]:
def get_cod_localidad(localidad_name):
    return [key  for (key, value) in localidadCodDictionaryNuse.items() if value == localidad_name][0]

In [20]:
def rebuild_location_in_siedco(original_df, index, driver):
    address = original_df.at[index,'DIRECCION_DEPURADA']
    address = re.sub(' +', ' ', original_df.at[index,'DIRECCION_DEPURADA']).strip()
    print(address)
    result_ws = ws_address_siedco.web_scrap_address(driver,address)
    print(result_ws)

    if result_ws != "Not found":        
        parsed_result = parse_address_ws(result_ws)
        print(parsed_result)
        if parsed_result["Dirección ingresada"] != address:
            return "Error loading address"
        else:            
            original_df.at[index,'LATITUD_Y'] = float(parsed_result['Latitud'])
            original_df.at[index,'LONGITUD_X'] = float(parsed_result['Longitud'])
            parsed_localidad = parsed_result['Localidad']
            if parsed_localidad == 'ANTONIO NARIÑO':
                original_df.at[index,'LOCALIDAD'] = parsed_localidad
            else:
                original_df.at[index,'LOCALIDAD'] = unidecode.unidecode(parsed_localidad)
            original_df.at[index,'COD_LOCALIDAD'] = int(get_cod_localidad(original_df.at[index,'LOCALIDAD']))
            original_df.at[index,'SEC_CATASTRAL'] = parsed_result['Sector catastral']
            assign_upz(original_df,index,parsed_result['UPZ'])
            return "Rebuilt"
    else:
        return "Not processed"


## Implement rebuild methods

### Rebuild location through address

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/02. rebuild_categorical_vars_siedco_05032020.csv'
df_input = pd.read_csv(data_location,delimiter=",")

In [ ]:
pd.DataFrame({"Tipo de dato":df_input.dtypes.values,
              "Celdas con valor '-'":(df_input == '-').sum().values,
              "Celdas con valor ''":(df_input == '').sum().values,
              "Celdas con valor ' '":(df_input == ' ').sum().values,
              "Celdas vacías": df_input.isna().sum().values},
             index=df_input.columns)

# Process small dataset to enhance web scraping performance

## Get original idx list to process

In [9]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/02. rebuild_categorical_vars_siedco_05032020.csv'
data = pd.read_csv(data_location,delimiter=",")

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (32,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df_empty_upz = data.loc[(data['COD_UPZ'] == '-')| (data.COD_UPZ.isna())]
df_empty_seccat = data.loc[data['COD_SEC_CATAST'] == '-']
df_empty_barrio = data.loc[data['COD_BARRIO'] == '-']
df_empty_upz.equals(df_empty_seccat) & df_empty_seccat.equals(df_empty_barrio)

True

In [11]:
df_empty_location_vars = df_empty_upz

In [12]:
len(df_empty_location_vars)

15764

In [13]:
df_empty_location_vars = df_empty_location_vars.loc[(df_empty_location_vars['DIRECCION_DEPURADA']!='-') & ~(df_empty_location_vars.DIRECCION_DEPURADA.isna())]

In [14]:
len(df_empty_location_vars)

15666

In [15]:
pd.DataFrame(((df_empty_location_vars == '-') | (df_empty_location_vars.isna())).sum()).T

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,ID_ORIGINAL,HECHO,CUENTA_HECHOS,CARACTERISTICA,ARMA_EMPLEADA,MOVIL_AGRESOR,MOVIL_VICTIMA,PAIS_PERSONA,MUNICIPIO_PERSONA,ESTADO_CIVIL_PERSONA,GRADO_INSTRUCCION_PERSONA,GENERO_PERSONA,EDAD_PERSONA,RANGO_VITAL,RANGO_EDAD,FECHA_HECHO,ANIO,SEMESTRE,TRIMESTRE,MES,SEMANA_ANIO,DIA,DIA_SEMANA,FESTIVO,HORA_HECHO,HORA,RANGO_HORA,RANGO_DEL_DIA,COD_LOCALIDAD,LOCALIDAD,COD_UPZ,UPZ,COD_SEC_CATAST,SEC_CATASTRAL,JURIS_ESTACION_AREA,ZONA,CLASE_SITIO,DIRECCION,LONGITUD_X,LATITUD_Y,CLAVE_ZONA,INDICE_LOCALIDAD,INDICE_CIUDAD,FECHA_CORTE,COD_BARRIO,BARRIO,STR_ESTADO_REG,DELITO_CARGO_PERSONA,MEDIO_CONOCIMIENTO,GRUPO_CLASE_SITIO,GRUPO_CLASE_SITIO_IMPUTADO,DIRECCION_DEPURADA
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15666,15666,15666,15666,0,0,0,0,0,0,0,0,0,0,15666,15666,0,0,0,3,0,0


In [16]:
#Try to rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' through address
df_empty_locations_with_address = df_empty_location_vars
list_idx_rebuild_location = list(df_empty_locations_with_address.index.values)
len(list_idx_rebuild_location)

15666

## Iterate over small datasets

In [17]:
import time
from datetime import datetime

In [18]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/02. rebuild_categorical_vars_siedco_05032020.csv'
df_input = pd.read_csv(data_location,delimiter=",")

In [21]:
df_output = df_input
idx_error_loading_address = []
num_reg_by_iter = 1000
registers_to_process = 5*num_reg_by_iter
rebuilt_registers = 0
registers_not_processed = 0
other_condition_counter = 0

file_path = '/Users/anamaria/Desktop/dev/security_project/datasets/'

for i in range(0, 5):
    first_reg = i*num_reg_by_iter
    last_reg = ((i+1)*num_reg_by_iter)-1
    print(first_reg,' to ', last_reg)
    current_list_idx_rebuild = list_idx_rebuild_location[first_reg:last_reg]
    
    #Rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' using web scraping  
    url='https://mapas.bogota.gov.co'
    driver = ws_address_siedco.web_scrap_page(url)

    for index in current_list_idx_rebuild:
        state = rebuild_location_in_siedco(df_output, index, driver)
        time.sleep(5)
        driver.refresh()
        time.sleep(10)
        if state == "Rebuilt":
            rebuilt_registers += 1
        elif state == "Not processed":
            registers_not_processed += 1
        elif state == "Error loading address":
            idx_error_loading_address.append(index)
        else:
            other_condition_counter += 1

        print('Rebuilt registers: ',rebuilt_registers,'/',registers_to_process)
        print('Registers not processed: ',registers_not_processed, '/', registers_to_process)
        print('Processing failures: ',len(idx_error_loading_address), '/', registers_to_process)
        
    time.sleep(30)
    # Save df output
    string_date = datetime.today().strftime('%d%m%Y')
    str_reg = str(first_reg) +'_to_'+ str(last_reg)
    file_name = "'" + file_path + 'rebuild_locations_siedco_idx_' + str_reg + '_' + string_date + ".csv'"
    df_output.to_csv(eval('r'+file_name),index=None)
    
    time.sleep(30)
    driver.quit()

0  to  999
CL 66 17 J 37 SUR
Dirección ingresada: CL 66 17 J 37 SURDirección encontrada: CL 66 S 17K 37Tipo dirección: Aproximación por mallaCódigo postal: 111961Sector catastral: LUCERO DEL SURUPZ: LUCERO (UPZ67)Localidad: CIUDAD BOLÍVARLatitud: 4.55221571392479Longitud: -74.1416034367497CHIP: Consultar CHIP asociados al lote   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'CL 66 17 J 37 SUR', 'Dirección encontrada': 'CL 66 S 17K 37', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '111961', 'Sector catastral': 'LUCERO DEL SUR', 'UPZ': 'LUCERO (UPZ67)', 'Localidad': 'CIUDAD BOLÍVAR', 'Latitud': '4.55221571392479', 'Longitud': '-74.1416034367497'}
Rebuilt registers:  1 / 5000
Registers not processed:  0 / 5000
Processing failures:  0 / 5000
CL 151 BBIS 115 32
Dirección ingresada: CL 151 BBIS 115 32Dirección encontrada: CL 151B BIS 115 32Tipo dirección: Asignada por CatastroCódigo postal: 111161Sector catastral: RINCON DE SANTA INESUPZ:

Rebuilt registers:  11 / 5000
Registers not processed:  0 / 5000
Processing failures:  0 / 5000
CL 58 A 18 B 12 SUR
Dirección ingresada: CL 58 A 18 B 12 SURDirección encontrada: CL 58A S 18B 12Tipo dirección: Asignada por CatastroCódigo postal: 110621Sector catastral: SAN BENITOUPZ: TUNJUELITO (UPZ62)Localidad: TUNJUELITOLatitud: 4.56382130600008Longitud: -74.1347226469999CHIP: Consultar CHIP asociados al lote   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'CL 58 A 18 B 12 SUR', 'Dirección encontrada': 'CL 58A S 18B 12', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '110621', 'Sector catastral': 'SAN BENITO', 'UPZ': 'TUNJUELITO (UPZ62)', 'Localidad': 'TUNJUELITO', 'Latitud': '4.56382130600008', 'Longitud': '-74.1347226469999'}
Rebuilt registers:  12 / 5000
Registers not processed:  0 / 5000
Processing failures:  0 / 5000
DG 46 G 14 B 23 SUR
Dirección ingresada: DG 46 G 14 B 23 SURDirección encontrada: DG 46 BIS S 13M 23Tipo dirección

Rebuilt registers:  22 / 5000
Registers not processed:  0 / 5000
Processing failures:  0 / 5000
KR 67 62 SUR
Dirección ingresada: KR 67 62 SURDirección encontrada: KR 67 62 20 STipo dirección: Aproximación por placaCódigo postal: 111911Sector catastral: MADELENAUPZ: ARBORIZADORA (UPZ65)Localidad: CIUDAD BOLÍVARLatitud: 4.59097261400007Longitud: -74.1574410269999CHIP: Consultar CHIP asociados al lote   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'KR 67 62 SUR', 'Dirección encontrada': 'KR 67 62 20 S', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111911', 'Sector catastral': 'MADELENA', 'UPZ': 'ARBORIZADORA (UPZ65)', 'Localidad': 'CIUDAD BOLÍVAR', 'Latitud': '4.59097261400007', 'Longitud': '-74.1574410269999'}
Rebuilt registers:  23 / 5000
Registers not processed:  0 / 5000
Processing failures:  0 / 5000
CL 42 78 J 9 SUR
Dirección ingresada: CL 42 78 J 9 SURDirección encontrada: CL 42 S 78J 09Tipo dirección: Asignada por CatastroCó

Rebuilt registers:  33 / 5000
Registers not processed:  0 / 5000
Processing failures:  0 / 5000
CL 77 A 82 . 34 SUR
Dirección ingresada: CL 77 A 82 34 SURDirección encontrada: DG 77A S 82 41Tipo dirección: Aproximación por placaCódigo postal: 111961Sector catastral: CEDRITOS DEL SURUPZ: EL TESORO (UPZ68)Localidad: CIUDAD BOLÍVARLatitud: 4.53932335300004Longitud: -74.1464756379999   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'CL 77 A 82 34 SUR', 'Dirección encontrada': 'DG 77A S 82 41', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '111961', 'Sector catastral': 'CEDRITOS DEL SUR', 'UPZ': 'EL TESORO (UPZ68)', 'Localidad': 'CIUDAD BOLÍVAR', 'Latitud': '4.53932335300004', 'Longitud': '-74.1464756379999'}
Rebuilt registers:  33 / 5000
Registers not processed:  0 / 5000
Processing failures:  1 / 5000
KR 78 A 66 38 SUR
Dirección ingresada: KR 78 A 66 38 SURDirección encontrada: KR 78A 66 38 STipo dirección: Asignada por CatastroCódigo po

Rebuilt registers:  42 / 5000
Registers not processed:  0 / 5000
Processing failures:  2 / 5000
KR 79 F 48 A 15 SUR
Dirección ingresada: KR 79 F 48 A 15 SURDirección encontrada: KR 79F 48A 15 STipo dirección: Asignada por CatastroCódigo postal: 110861Sector catastral: CASABLANCAUPZ: KENNEDY CENTRAL (UPZ47)Localidad: KENNEDYLatitud: 4.61770071100005Longitud: -74.1692932919999CHIP: Consultar CHIP asociados al lote   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'KR 79 F 48 A 15 SUR', 'Dirección encontrada': 'KR 79F 48A 15 S', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '110861', 'Sector catastral': 'CASABLANCA', 'UPZ': 'KENNEDY CENTRAL (UPZ47)', 'Localidad': 'KENNEDY', 'Latitud': '4.61770071100005', 'Longitud': '-74.1692932919999'}
Rebuilt registers:  43 / 5000
Registers not processed:  0 / 5000
Processing failures:  2 / 5000
CL 86 15
Dirección ingresada: CL 86 15Dirección encontrada: CL 86A BIS 15 22Tipo dirección: Aproximación por m

...

Rebuilt registers:  1108 / 5000
Registers not processed:  74 / 5000
Processing failures:  18 / 5000
KR 98 B 73 15 SUR
Dirección ingresada: KR 98 B 73 15 SURDirección encontrada: KR 98B 73 15 STipo dirección: Asignada por CatastroCódigo postal: 110721Sector catastral: CIUDADELA EL RECREOUPZ: TINTAL SUR (UPZ87)Localidad: BOSALatitud: 4.63295602600004Longitud: -74.2059086869999CHIP: Consultar CHIP asociados al lote   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'KR 98 B 73 15 SUR', 'Dirección encontrada': 'KR 98B 73 15 S', 'Tipo dirección': 'Asignada por Catastro', 'Código postal': '110721', 'Sector catastral': 'CIUDADELA EL RECREO', 'UPZ': 'TINTAL SUR (UPZ87)', 'Localidad': 'BOSA', 'Latitud': '4.63295602600004', 'Longitud': '-74.2059086869999'}
Rebuilt registers:  1109 / 5000
Registers not processed:  74 / 5000
Processing failures:  18 / 5000
CL 65 B 77 L SUR
Dirección ingresada: CL 65 B 77 L SURDirección encontrada: CL 65B BIS S 77L 0Tipo dirección

Rebuilt registers:  1119 / 5000
Registers not processed:  75 / 5000
Processing failures:  18 / 5000
CL 31 5 SUR
Dirección ingresada: CL 31 5 SURDirección encontrada: CL 31 S 5A 07Tipo dirección: Aproximación por mallaCódigo postal: 110421Sector catastral: SAN ISIDROUPZ: 20 DE JULIO (UPZ34)Localidad: SAN CRISTÓBALLatitud: 4.56542120100005Longitud: -74.0983913169999   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'CL 31 5 SUR', 'Dirección encontrada': 'CL 31 S 5A 07', 'Tipo dirección': 'Aproximación por malla', 'Código postal': '110421', 'Sector catastral': 'SAN ISIDRO', 'UPZ': '20 DE JULIO (UPZ34)', 'Localidad': 'SAN CRISTÓBAL', 'Latitud': '4.56542120100005', 'Longitud': '-74.0983913169999'}
Rebuilt registers:  1120 / 5000
Registers not processed:  75 / 5000
Processing failures:  18 / 5000
DG 67 77 33 SUR
Dirección ingresada: DG 67 77 33 SURDirección encontrada: DG 67 S 77 41Tipo dirección: Aproximación por placaCódigo postal: 111951Sector catastral

Rebuilt registers:  1130 / 5000
Registers not processed:  75 / 5000
Processing failures:  18 / 5000
DG 46 49 51 SUR
Dirección ingresada: DG 46 49 51 SURDirección encontrada: DG 46 S 49 51Tipo dirección: Dirección AntiguaCódigo postal: 110611Sector catastral: VENECIAUPZ: VENECIA (UPZ42)Localidad: TUNJUELITOLatitud: 4.59335717239249Longitud: -74.1361953329745CHIP: Consultar CHIP asociados al lote   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'DG 46 49 51 SUR', 'Dirección encontrada': 'DG 46 S 49 51', 'Tipo dirección': 'Dirección Antigua', 'Código postal': '110611', 'Sector catastral': 'VENECIA', 'UPZ': 'VENECIA (UPZ42)', 'Localidad': 'TUNJUELITO', 'Latitud': '4.59335717239249', 'Longitud': '-74.1361953329745'}
Rebuilt registers:  1131 / 5000
Registers not processed:  75 / 5000
Processing failures:  18 / 5000
CL 39 A 79 G 28 SUR
Dirección ingresada: CL 39 A 79 G 28 SURDirección encontrada: CL 39A S 79G 28Tipo dirección: Asignada por CatastroCódigo p

Rebuilt registers:  1141 / 5000
Registers not processed:  75 / 5000
Processing failures:  18 / 5000
CL 3 9 80
Dirección ingresada: CL 3 9 80Dirección encontrada: CL 3 9 70Tipo dirección: Aproximación por placaCódigo postal: 110321Sector catastral: SANTA BARBARAUPZ: LAS CRUCES (UPZ95)Localidad: SANTA FELatitud: 4.59219538700006Longitud: -74.0823146479999CHIP: Consultar CHIP asociados al lote   Mostrar información asociada   Mostrar información asociada
{'Dirección ingresada': 'CL 3 9 80', 'Dirección encontrada': 'CL 3 9 70', 'Tipo dirección': 'Aproximación por placa', 'Código postal': '110321', 'Sector catastral': 'SANTA BARBARA', 'UPZ': 'LAS CRUCES (UPZ95)', 'Localidad': 'SANTA FE', 'Latitud': '4.59219538700006', 'Longitud': '-74.0823146479999'}
Rebuilt registers:  1142 / 5000
Registers not processed:  75 / 5000
Processing failures:  18 / 5000
KR 82 H 61 12 SUR
Not found
Rebuilt registers:  1142 / 5000
Registers not processed:  76 / 5000
Processing failures:  18 / 5000
KR 87 A 69 A 98


TimeoutException: Message: 


### print(rebuilt_registers)
print(registers_not_processed)
print(other_condition_counter)

In [ ]:
#save lists
MyFile=open('/Users/anamaria/Desktop/dev/security_project/datasets/idx_error_loading_address_siedco_05032020.txt','w')
MyList=map(lambda x: str(x)+', ', idx_error_loading_address)
MyFile.writelines(MyList)
MyFile.close()

In [ ]:
df_output.to_csv(r'/home/combios/Documents/amreyesp/rebuild_locations_nuse_idx_39kto48k_29012020.csv',index=None)

In [ ]:
#iterate over idx_error_loading_address to rebuild location if necesary